In [15]:
import pandas as pd
import numpy as np
import re

In [16]:
# import file
file = pd.read_csv('format.csv',header=None,error_bad_lines=False)

In [17]:
# function for removing special characters from string
def preprocess(string):
    string = re.sub('\W+',' ', string)
    return string

In [18]:
# create variables
platform = preprocess(file.iloc[0,1])
previoustitle = preprocess(file.iloc[1,1])
previoussub = preprocess(file.iloc[2,1])
newtitle = preprocess(file.iloc[3,1])
newsub = preprocess(file.iloc[4,1])
listandranks = file.iloc[7:1000,0:3]
previousranks = file.iloc[7:1000,1]
newranks = file.iloc[7:1000,2]

In [19]:
# concatenate previous and new titles and make to lower case
previous = np.concatenate((previoustitle.split(), previoussub.split()))
previous = [x.lower() for x in previous]
new = np.concatenate((newtitle.split(), newsub.split()))
new = [x.lower() for x in new]

In [20]:
# make list of keywords removed and keywords added
keywordsremoved = list(set(previous) - set(new))
keywordsadded = list(set(new) - set(previous))

In [23]:
# iterate through keyword lists, check what the change in rank has been for the ranks of targets KWs containing that list & append to DF
avgdif = pd.DataFrame(columns = ['Keyword', 'Average Difference'])
for i in keywordsadded:
    compare = listandranks[listandranks[0].str.contains(i)]
    if not compare.empty:
        compare = compare.fillna(0)
        compare = compare[(compare[[1,2]] != 0).all(axis=1)]
        compare = compare.astype(dtype={1:'int', 2:'int'}, copy=True, errors='raise')
        compare['Difference'] = compare.iloc[:,1] - compare.iloc[:,2]
        mean = int(compare['Difference' !=0].mean())
        avgdif = avgdif.append([{'Keyword': i, 'Average Difference': mean}])

avgdec = pd.DataFrame(columns = ['Keyword', 'Average Difference'])

for i in keywordsremoved:
    compare = listandranks[listandranks[0].str.contains(i)]
    if not compare.empty:
        compare = compare.fillna(0)
        compare = compare[(compare[[1,2]] != 0).all(axis=1)]
        compare = compare.astype(dtype={1:'int', 2:'int'}, copy=True, errors='raise')
        compare['Difference'] = compare.iloc[:,1] - compare.iloc[:,2]
        mean = int(compare['Difference'!=0].mean())
        avgdec = avgdec.append([{'Keyword': i, 'Average Difference': mean}])

In [24]:
# combine keywords added + removed 
final = pd.concat([avgdif, avgdec])
final.reset_index(drop=True)

print(final)

    Keyword Average Difference
0    better                102
0       how                 51
0   anxiety                300
0   relieve                300
0  meditate                 32
0    stress                300
0    guided                 22
0    sounds                300


In [25]:
# send to CSV
final.to_csv('output.csv')